In [28]:
import pandas as pd
import numpy as np  

In [ ]:
hotel_city_chanel_combin_extract  = pd.read_csv('c:\\Users\\paulo\\Desktop\\hotels brazil\\data\\other\\hotel_city_chanel_combin_extract.csv')
hotel_city_chanel_combin_extract

In [ ]:
hotel_city_chanel_combin_extract  = pd.read_csv('c:\\Users\\paulo\\Desktop\\hotels brazil\\data\\other\\hotel_city_chanel_combin_extract.csv')
hotel_city_chanel_combin_extract

In [ ]:
def get_unseen_hotels(df, canal_ids):
    if isinstance(canal_ids, int):
        canal_ids = [canal_ids]
    
    result = {}
    
    for canal_id in canal_ids:
        # Get the Cidade_IDs corresponding to the given Canal_ID
        cidade_ids = df[df['Canal_ID'] == canal_id]['Cidade_ID'].unique()
        
        # Get all Hotel_IDs in the corresponding Cidade_IDs
        all_hotels_in_cidade = df[df['Cidade_ID'].isin(cidade_ids)]['Hotel_ID'].unique()
        
        # Get the Hotel_IDs already seen by the given Canal_ID
        seen_hotels = df[df['Canal_ID'] == canal_id]['Hotel_ID'].unique()
        
        # Get the Hotel_IDs not already seen by the given Canal_ID
        unseen_hotels = [hotel for hotel in all_hotels_in_cidade if hotel not in seen_hotels]
        
        result[canal_id] = unseen_hotels
    
    return result

# Example usage:
unseen_hotels = get_unseen_hotels(hotel_city_chanel_combin_extract, [124])
print(unseen_hotels)

In [ ]:
hotel_city_chanel_combin_extract[(hotel_city_chanel_combin_extract['Cidade_ID'] == 1571478) & (hotel_city_chanel_combin_extract['Hotel_ID'] == 13211)]


In [ ]:
hotel_city_chanel_combin_extract[(hotel_city_chanel_combin_extract['Canal_ID'] == 124) & (hotel_city_chanel_combin_extract['Cidade_ID'] == 1571478)]

In [ ]:
unseen_hotels_df = pd.DataFrame.from_dict(unseen_hotels, orient='index').transpose()
unseen_hotels_df

In [139]:

ix = hotel_city_chanel_combin_extract[hotel_city_chanel_combin_extract['Hotel_ID'].isin([16573])]['Canal_ID'].unique()

In [ ]:
# Split the 'distinct_combinations' column into two separate columns
data_lake_prd_314410_cz_hoteis_canais_combinations[['Hotel_ID', 'Canal_ID']] = data_lake_prd_314410_cz_hoteis_canais_combinations['distinct_combinations'].str.split('-', expand=True)

# Convert the new columns to integers
data_lake_prd_314410_cz_hoteis_canais_combinations['Hotel_ID'] = data_lake_prd_314410_cz_hoteis_canais_combinations['Hotel_ID'].astype(int)
data_lake_prd_314410_cz_hoteis_canais_combinations['Canal_ID'] = data_lake_prd_314410_cz_hoteis_canais_combinations['Canal_ID'].astype(int)

# Drop the original 'distinct_combinations' column
data_lake_prd_314410_cz_hoteis_canais_combinations.drop(columns=['distinct_combinations'], inplace=True)
data_lake_prd_314410_cz_hoteis_canais_combinations

In [ ]:
data_lake_prd_314410_cz_hoteis = pd.read_csv('c:\\Users\\paulo\\Desktop\\hotels brazil\\data\\lookups\\data-lake-prd-314410.cz.hoteis.csv')
data_lake_prd_314410_cz_hoteis

In [32]:
data_lake_prd_314410_cz_hoteis = data_lake_prd_314410_cz_hoteis[['Hotel_ID', 'Estrelas', 'Quartos', 'CategoriaHotel', 'Moeda', 'Cidade_ID']]

In [ ]:
data_lake_prd_314410_cz_hoteis

In [ ]:
data_lake_prd_314410_cz_cidades = pd.read_csv('c:\\Users\\paulo\\Desktop\\hotels brazil\\data\\lookups\\data-lake-prd-314410.cz.cidades.csv')
data_lake_prd_314410_cz_cidades

In [35]:
data_lake_prd_314410_cz_cidades = data_lake_prd_314410_cz_cidades[['Cidade_ID', 'Pais']]

In [ ]:
merged_data = pd.merge(data_lake_prd_314410_cz_hoteis, 
                       data_lake_prd_314410_cz_cidades, 
                       on='Cidade_ID', 
                       how='left')
merged_data

In [ ]:
country_counts = merged_data.groupby('Pais').size().reset_index(name='Counts').sort_values(by='Counts', ascending=False)
print(country_counts)

In [ ]:
country_counts['Counts'].sum()

In [ ]:
19397/19650  # 1.3% of the hotels cannot be mapped to a country ?

In [ ]:
merged_data

In [ ]:
merged_data.dropna(subset=['Pais'], inplace=True)
merged_data.shape

In [ ]:
merged_data.drop(columns=['Cidade_ID'], inplace=True)
merged_data

In [ ]:
merged_data

In [44]:
data_lake_prd_314410_cz_pull_pesquisas_average_price_hotel_currency = pd.read_csv('c:\\Users\\paulo\\Desktop\\hotels brazil\\data\\other\\data-lake-prd-314410.cz.pull-pesquisas_average_price_hotel_currency.csv')
data_lake_prd_314410_cz_moedas = pd.read_csv('c:\\Users\\paulo\\Desktop\\hotels brazil\\data\\lookups\\data-lake-prd-314410.cz.moedas.csv')

In [ ]:
# Perform left join
data_lake_prd_314410_cz_pull_pesquisas_average_price_hotel_currency = pd.merge(data_lake_prd_314410_cz_pull_pesquisas_average_price_hotel_currency, 
                     data_lake_prd_314410_cz_moedas[['Moeda_ID', 'Cotacao_USD']], 
                     on='Moeda_ID', 
                     how='left')

# Display the merged dataframe
print(data_lake_prd_314410_cz_pull_pesquisas_average_price_hotel_currency.head())

In [46]:
data_lake_prd_314410_cz_pull_pesquisas_average_price_hotel_currency['weighted_mean_diaria_usd'] = \
    data_lake_prd_314410_cz_pull_pesquisas_average_price_hotel_currency['weighted_mean_diaria'] * \
        data_lake_prd_314410_cz_pull_pesquisas_average_price_hotel_currency['Cotacao_USD']

In [ ]:
data_lake_prd_314410_cz_pull_pesquisas_average_price_hotel_currency

In [ ]:
merged_data = pd.merge(merged_data, 
                     data_lake_prd_314410_cz_pull_pesquisas_average_price_hotel_currency[['Hotel_ID', 'Cotacao_USD', 'weighted_mean_diaria_usd']], 
                     on='Hotel_ID', 
                     how='left')

merged_data

In [49]:
data_lake_prd_314410_cz_pull_pesquisas_estadia_x_reservas_volume = pd.read_csv('c:\\Users\\paulo\\Desktop\\hotels brazil\\data\\other\\data-lake-prd-314410.cz.pull-pesquisas_estadia_x_reservas_volume.csv')


In [ ]:
data_lake_prd_314410_cz_pull_pesquisas_estadia_x_reservas_volume

In [ ]:
merged_data = pd.merge(merged_data, 
                     data_lake_prd_314410_cz_pull_pesquisas_estadia_x_reservas_volume, 
                     on='Hotel_ID', 
                     how='left')

merged_data

In [ ]:
data_lake_prd_314410_cz_pull_pesquisas_estadia_x_reservas_x_price_receitas = pd.read_csv('c:\\Users\\paulo\\Desktop\\hotels brazil\\data\\other\\data-lake-prd-314410.cz.pull-pesquisas_estadia_x_reservas_x_price_receitas.csv')
data_lake_prd_314410_cz_pull_pesquisas_estadia_x_reservas_x_price_receitas

In [ ]:
merged_data = pd.merge(merged_data, 
                     data_lake_prd_314410_cz_pull_pesquisas_estadia_x_reservas_x_price_receitas, 
                     on='Hotel_ID', 
                     how='left')

merged_data

In [ ]:
merged_data['total_reservas_estadia_diaria'] = \
    merged_data['total_reservas_estadia_diaria'] * \
        merged_data['Cotacao_USD']

merged_data

In [ ]:
merged_data.drop(columns=['Moeda'], inplace=True)
merged_data

In [ ]:
merged_data.drop(columns=['Cotacao_USD'], inplace=True)
merged_data

In [ ]:
merged_data.dropna(inplace=True)
merged_data

In [60]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt


In [ ]:
# Select relevant columns
numerical_cols = ['Estrelas', 'Quartos', 'weighted_mean_diaria_usd', 
                  'total_reservas_estadia', 'total_reservas_estadia_diaria']
categorical_cols = ['CategoriaHotel', 'Pais']

# Normalize numerical variables
scaler = StandardScaler()
data_scaled = scaler.fit_transform(merged_data[numerical_cols])

# Encode categorical variables
data_encoded = pd.get_dummies(merged_data[categorical_cols], drop_first=True)

# Combine normalized and encoded data
data_final = pd.concat([pd.DataFrame(data_scaled, columns=numerical_cols), data_encoded.reset_index(drop=True)], axis=1)


In [ ]:
# Determine optimal number of clusters using the elbow method
wcss = []
for i in range(2, 10):
    kmeans = KMeans(n_clusters=i, random_state=42)
    kmeans.fit(data_final)
    wcss.append(kmeans.inertia_)

plt.plot(range(2, 10), wcss, marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.title('Elbow Method')
plt.show()

In [ ]:
# Choose optimal number of clusters (e.g., k=4)
kmeans = KMeans(n_clusters=10, random_state=42)
merged_data['Cluster'] = kmeans.fit_predict(data_final)

In [ ]:
# Analyze contributions using PCA
pca = PCA(n_components=10)
pca_data = pca.fit_transform(data_final)
plt.scatter(pca_data[:, 2], pca_data[:, 4], c=merged_data['Cluster'], cmap='viridis')
plt.title('PCA of Clusters')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.show()

In [ ]:
# Analyze cluster centroids
cluster_centers = pd.DataFrame(kmeans.cluster_centers_, columns=data_final.columns)
important_vars = cluster_centers.T
important_vars.columns = [f'Cluster_{i}' for i in range(10)]
print(important_vars)

In [ ]:
to_export = important_vars
to_export.tocsv('c:\\Users\\paulo\\Desktop\\hotels brazil\\data\\output\\clusters.csv')

In [81]:
to_export.to_csv('c:\\Users\\paulo\\Desktop\\hotels brazil\\data\\other\\clusters.csv')

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

def preprocess_data(data):
    """
    Preprocess the data to create a numerical feature matrix.
    - Scales numerical columns
    - One-hot encodes categorical columns
    - Converts boolean columns to numerical
    """
    # Define column groups
    numerical_cols = ['Estrelas', 'Quartos', 'weighted_mean_diaria_usd', 
                      'total_reservas_estadia', 'total_reservas_estadia_diaria']
    categorical_cols = ['CategoriaHotel', 'Pais']
    boolean_cols = data.select_dtypes(include=['bool']).columns.tolist()
    
    # Convert boolean columns to integers
    data[boolean_cols] = data[boolean_cols].astype(int)
    
    # Scale numerical columns
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(data[numerical_cols])
    data_scaled_df = pd.DataFrame(data_scaled, columns=numerical_cols, index=data.index)
    
    # One-hot encode categorical columns
    data_encoded = pd.get_dummies(data[categorical_cols], drop_first=True)
    
    # Combine scaled numerical, one-hot encoded categorical, and boolean features
    features = pd.concat([data_scaled_df, data_encoded, data[boolean_cols]], axis=1)
    return features

In [96]:
from scipy.spatial.distance import cdist

def rank_similar_hotels(hotel_id, data, features, metric='euclidean'):
    """
    Rank hotels based on similarity to a given Hotel_ID.
    """
    # Check if the Hotel_ID exists
    if hotel_id not in data['Hotel_ID'].values:
        raise ValueError(f"Hotel_ID {hotel_id} not found in the dataset.")
    
    # Get the index of the given Hotel_ID
    idx = data.index[data['Hotel_ID'] == hotel_id][0]
    
    # Compute pairwise distances
    distances = cdist([features.iloc[idx]], features, metric=metric)[0]
    
    # Add distances to the original dataset
    data_copy = data.copy()
    data_copy['Similarity'] = distances
    
    # Sort by similarity and return ranked hotels
    ranked = data_copy.sort_values('Similarity').reset_index(drop=True)
    return ranked[['Hotel_ID', 'Similarity']]


In [111]:
def rank_similar_hotels_cosine(hotel_id, data, features):
    """
    Rank hotels based on cosine similarity to a given Hotel_ID.
    """
    # Ensure the Hotel_ID exists
    if hotel_id not in data['Hotel_ID'].values:
        raise ValueError(f"Hotel_ID {hotel_id} not found in the dataset.")
    
    # Get the index of the given Hotel_ID
    idx = data.index[data['Hotel_ID'] == hotel_id][0]
    
    # Compute cosine similarity
    similarity_scores = cosine_similarity([features.iloc[idx]], features)[0]
    
    # Add similarity scores to the original dataset
    data_copy = data.copy()
    data_copy['Similarity'] = similarity_scores
    
    # Sort by similarity and return ranked results
    ranked = data_copy.sort_values('Similarity', ascending=False).reset_index(drop=True)
    return ranked[['Hotel_ID', 'Similarity']]

In [ ]:
# Sample Data
data = merged_data

# Step 1: Preprocess the data
features = preprocess_data(data)

# Step 2: Rank hotels similar to Hotel_ID 3728 using cosine similarity
ranked_hotels = rank_similar_hotels_cosine(3728, data, features)

# Step 3: Print results
print(ranked_hotels)


In [ ]:
merged_data[merged_data['Hotel_ID'].isin([3328, 9385, 17731])]


In [ ]:
merged_data[merged_data['Cluster'].isin([0])]
